# DEPRECATED

Don't use this any  more

See `metric_querier.py`

# Metrics of classification probabilities

This is the setup notebook.  It logs into the TOM, defines function `run_query`, and defines dictionaries `classifier_info` and `classname`

Include this in notebooks that muck about with classification probabilities with

```%run query_for_probabilistic_metrics_setup.ipynb```

Some key views in the database:

**elasticc_view_classifications_probmetrics** has columns "classifierId", "classId", "trueclassid", "tbin", "probbin", and "count".  For a given combination of classifier/class/true class, it is the 2d histogram with respect to probability and time of classifications.  Note that there are some duplicates in the database; about 5% of classifier/source combinations have total probability > 1.001.  The bins are defined using the postgres width_bucket function:
  > `tbin = width_bucket(s."midPointTai" - ot.peakmjd, -30, 100, 26)`
  
  > `probbin = width_bucket(c.probability, 0, 1, 20)`
  
Postgres starts counting bins at 1, so the meanings of probbins are 0: probability<0, 1: probability [0,0.5), 2: probability [0.05, 0.1), ..., 20: probability [0.95,1.0), 21: probability>=1.  This does mean we'll probably want to lump bin 21 with bin 20, otherwise we'll lose things with 1.0 probability.

For tbins, 0: Δt<30, 1: Δt [-30, -25), 2: Δt: [-25, 20), ..., 26: Δt [-95, 100), 27: Δt>=100

**elasticc_view_classifications_totprobpersource** has columns "classifierId","diaSourceId", and "totprob".  This is one way of looking for duplicates.  It has the sum of all probabilities reported by a given classifier for a given source.  Where this is an integer >1, it means *either* a duplicate entry, or that the classifier redid the classification.  Where it's >1, it is still probably that.  In this table, 5.1% of entires are >1.001, and 22.5% of entries are <0.999.  We've decided for the ones that are <1, we take the classifiers at their word and consider the remainder to be "dunno".  For the ones >1, we haven't decided yet how to deal with it, and to first order are ignoring it, realizing that this means that there is likely a 5% corruption in our data that's almost certainly biased.

In [20]:
%matplotlib inline
import sys
import os
import requests
import datetime
import dateutil.parser
import json
import numpy
import pandas
import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics

In [21]:
 os.getenv("HOME")

'/global/homes/r/raknop'

In [22]:
url = "https://desc-tom.lbl.gov"
# url = "https://desc-tom-rknop-dev.lbl.gov"

# Somehow you need to set the "username" and "password" variables.
# Do NOT hardcode the password into any code that will be world
# readable.  You can use the python input statement, or, as I do
# here, read the password from a file that is only readable by
# you.  (This isn't perfect, but it's better than having the
# password sit around in a world readable file.)
username = "rknop"
with open( os.path.join( os.getenv("HOME"), "secrets", "tom_"+username+"_passwd" ) ) as ifp:
    password = ifp.readline().strip()

In [23]:
rqs = requests.session()
rqs.get( f'{url}/accounts/login/' )
res = rqs.post( f'{url}/accounts/login/',
                data={ "username": username,
                       "password": password,
                       "csrfmiddlewaretoken": rqs.cookies['csrftoken'] } )
if res.status_code != 200:
    raise RuntimeError( f"Failed to log in; http status: {res.status_code}" )
if 'Please enter a correct' in res.text:
    raise RuntimeError( "Failed to log in.  I think.  Put in a debug break and look at res.text" )
rqs.headers.update( { 'X-CSRFToken': rqs.cookies['csrftoken'] } )

In [29]:
tbin_min = -30
tbin_max = 100
tbin_num = 26
delta_tbin = ( tbin_max - tbin_min ) / tbin_num
def tbin_val( intbin ):
    tbin = numpy.atleast_1d( intbin )
    rval = numpy.array( tbin.shape )
    rval = ( tbin  - 1 ) * delta_tbin + tbin_min + delta_tbin  / 2.
    rval[ tbin==0 ] = tbin_min - delta_tbin / 2.
    rval[ tbin==tbin_num+1 ] = tbin_max + delta_tbin / 2.
    return rval[0] if numpy.isscalar( intbin ) else rval
    
probbin_min = 0
probbin_max = 1
probbin_num = 20
delta_probbin = ( probbin_max - probbin_min ) / probbin_num
def probbin_val( inprobbin ):
    probbin = numpy.atleast_1d( inprobbin )
    rval = numpy.array( probbin.shape )
    rval = ( probbin-1 ) * delta_probbin + delta_probbin / 2.
    rval[ probbin==0 ] = -1
    rval[ probbin==probbin_num+1 ] = 1
    return rval[0] if numpy.isscalar( inprobbin ) else rval

In [26]:
def run_query(query, subdict=None):
    if subdict == None:
        subdict = {}
    result = rqs.post( f'{url}/db/runsqlquery/',
        json={ 'query': query, 'subdict': subdict } )
    if result.status_code != 200:
        sys.stderr.write( f"ERROR: got status code {result.status_code} ({result.reason})\n" )
    else:
        data = json.loads( result.text )
        if ( 'status' not in data ) or ( data['status'] != 'ok' ):
            sys.stderr.write( "Got unexpected response\n" )
            print(data['error'])
        else:
            return data['rows']

In [27]:
rows = run_query( 'SELECT "classifierId","brokerName","brokerVersion",'
                  '"classifierName","classifierParams" '
                  'FROM elasticc_brokerclassifier' )
classifier_info = {}
for row in rows:
    classifier_info[row["classifierId"]] = row             
classifier_info

{36: {'classifierId': 36,
  'brokerName': 'AMPEL',
  'brokerVersion': 'v0.2',
  'classifierName': 'ElasticcLiveSNGuess',
  'classifierParams': 'XGBUnified+Parsnip04'},
 37: {'classifierId': 37,
  'brokerName': 'AMPEL',
  'brokerVersion': 'v0.2',
  'classifierName': 'ElasticcLive',
  'classifierParams': 'XGBUnified+Parsnip04'},
 38: {'classifierId': 38,
  'brokerName': 'AMPEL',
  'brokerVersion': 'v0.2',
  'classifierName': 'ElasticcLiveSNGuessParsnip',
  'classifierParams': 'XGBUnified+Parsnip04'},
 39: {'classifierId': 39,
  'brokerName': 'AMPEL',
  'brokerVersion': 'v0.2',
  'classifierName': 'ElasticcLiveSNGuessParsnipPrior',
  'classifierParams': 'XGBUnified+Parsnip04'},
 40: {'classifierId': 40,
  'brokerName': 'ALeRCE',
  'brokerVersion': '3.1.0',
  'classifierName': 'balto',
  'classifierParams': '3.1.0'},
 41: {'classifierId': 41,
  'brokerName': 'ALeRCE',
  'brokerVersion': '1.0.0',
  'classifierName': 'messi',
  'classifierParams': '1.0.0'},
 42: {'classifierId': 42,
  'broke

In [18]:
rows = run_query( 'SELECT DISTINCT ON ("classId") "classId",description '
                  'FROM elasticc_gentypeofclassid '
                  'ORDER BY "classId"' )
classname = {}
for row in rows:
    classname[row["classId"]] = row["description"]

In [19]:
classname

{0: 'Static/Other',
 1: 'Non-Recurring',
 2: 'Recurring',
 10: 'Non-Recurring/Other',
 11: 'SN-like',
 12: 'Fast',
 13: 'Long',
 20: 'Recurring/Other',
 21: 'Periodic',
 22: 'Non-Periodic',
 110: 'SN-like/Other',
 111: 'Ia',
 112: 'Ib/c',
 113: 'II',
 114: 'Iax',
 115: '91bg',
 120: 'Fast/Other',
 121: 'KN',
 122: 'M-dwarf Flare',
 123: 'Dwarf Nova',
 124: 'uLens',
 130: 'Long/Other',
 131: 'SLSN',
 132: 'TDE',
 133: 'ILOT',
 134: 'CART',
 135: 'PISN',
 210: 'Periodic/Other',
 211: 'Cepheid',
 212: 'RR Lyrae',
 213: 'Delta Scuti',
 214: 'EB',
 215: 'LPV/Mira',
 220: 'Non-Periodic/Other',
 221: 'AGN'}